In [ ]:
pip install requests

In [ ]:
pip install pymysql

In [ ]:
pip install selenium

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pymysql
import time

# MySQL 연결
conn = pymysql.connect(
    host='localhost',
    user='root',
    password='4205',
    database='qnet_crawling2',
    charset='utf8mb4'
)
cursor = conn.cursor()

# 분야 저장
def save_field(field_id, depth1, depth2):
    cursor.execute("""
        INSERT INTO field (id, depth1, depth2)
        VALUES (%s, %s, %s)
        ON DUPLICATE KEY UPDATE depth1=%s, depth2=%s
    """, (field_id, depth1, depth2, depth1, depth2))
    conn.commit()

# 자격증 저장
def save_qualification(field_id, items):
    for name, jmCd in items:
        cursor.execute("""
            INSERT INTO qualification (name, JMCD, fieldId, course)
            VALUES (%s, %s, %s, %s)
            ON DUPLICATE KEY UPDATE name=%s, fieldId=%s
        """, (name, jmCd, field_id, None, name, field_id))
    conn.commit()

# 크롬 드라이버 실행
driver = webdriver.Chrome()
driver.get("https://q-net.or.kr/crf005.do#none")
wait = WebDriverWait(driver, 10)

print("🔍 크롤링 시작...\n")

# 1차 분야
all_first_level_links = wait.until(EC.presence_of_all_elements_located(
    (By.CSS_SELECTOR, "a[data-target^='duty_step2_']")))
first_level_links = [elem for elem in all_first_level_links if elem.is_displayed()]

for i in range(len(first_level_links)):
    all_first_level_links = wait.until(EC.presence_of_all_elements_located(
        (By.CSS_SELECTOR, "a[data-target^='duty_step2_']")))
    first_level_links = [elem for elem in all_first_level_links if elem.is_displayed()]

    first = first_level_links[i]
    first_text = first.text.strip()
    print(f"▶ 1차 클릭: {first_text}")
    driver.execute_script("arguments[0].click();", first)
    time.sleep(0.5)

    # 2차 분야
    try:
        all_second_level_links = wait.until(EC.presence_of_all_elements_located(
            (By.CSS_SELECTOR, "a[data-target^='duty_step3_']")))
        second_level_links = [elem for elem in all_second_level_links if elem.is_displayed()]
    except:
        continue

    for j in range(len(second_level_links)):
        all_second_level_links = wait.until(EC.presence_of_all_elements_located(
            (By.CSS_SELECTOR, "a[data-target^='duty_step3_']")))
        second_level_links = [elem for elem in all_second_level_links if elem.is_displayed()]

        second = second_level_links[j]
        second_text = second.text.strip()
        second_id = second.get_attribute("data-target")
        print(f"  ▶ 2차 클릭: {second_text} (id={second_id})")

        save_field(second_id, first_text, second_text)
        driver.execute_script("arguments[0].click();", second)
        time.sleep(0.5)

        # 자격증 수집
        try:
            third_links = WebDriverWait(driver, 5).until(
                EC.presence_of_all_elements_located(
                    (By.CSS_SELECTOR, "div.qualList.qulList_lv4 div.limitY div.item ul li"))
            )
            if third_links:
                items = []
                for li in third_links:
                    jmCd = li.find_element(By.CSS_SELECTOR, "input[name='jmcd']").get_attribute("value").strip()
                    name = li.find_element(By.CSS_SELECTOR, "input[name='jmNm']").get_attribute("value").strip()
                    items.append((name, jmCd))
                    print(f"      - {name} (jmCd={jmCd})")

                save_qualification(second_id, items)
        except:
            continue

    driver.get("https://q-net.or.kr/crf005.do#none")
    wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a[data-target^='duty_step2_']")))
    time.sleep(0.5)

print("\n✅ 크롤링 완료!")
driver.quit()
conn.close()


In [24]:
import requests
import xml.etree.ElementTree as ET
import pymysql

API_KEY = "9aa104a7dabd35bedd0c93bad373cd645655d30f4edfd6dc28f671e7b023fb48"

conn = pymysql.connect(
    host="localhost",
    user="root",
    password="4205",
    database="qnet_crawling2",
    charset="utf8mb4"
)
cursor = conn.cursor()

def to_datetime_or_none(value):
    if value is None or value.strip() == "":
        return None
    return value

def fetch_exam_schedule(jmCd, year="2025"):
    url = "http://apis.data.go.kr/B490007/qualExamSchd/getQualExamSchdList"
    params = {"serviceKey": API_KEY, "pageNo": "1", "numOfRows": "50", "dataFormat": "xml", "implYy": year, "jmCd": jmCd}
    response = requests.get(url, params=params)
    if response.status_code != 200:
        return []
    root = ET.fromstring(response.text)
    items = root.findall(".//item")

    schedules = []
    for item in items:
        schedules.append({
            "jmCd": jmCd,
            "year": item.findtext("implYy"),
            "period": item.findtext("implSeq"),
            "docRegStart": to_datetime_or_none(item.findtext("docRegStartDt")),
            "docRegEnd": to_datetime_or_none(item.findtext("docRegEndDt")),
            "docVacancyStart": to_datetime_or_none(item.findtext("docRegStartDt2")),
            "docVacancyEnd": to_datetime_or_none(item.findtext("docRegEndDt2")),
            "docExamStart": to_datetime_or_none(item.findtext("docExamStartDt")),
            "docExamEnd": to_datetime_or_none(item.findtext("docExamEndDt")),
            "docPass": to_datetime_or_none(item.findtext("docPassDt")),
            "pracRegStart": to_datetime_or_none(item.findtext("pracRegStartDt")),
            "pracRegEnd": to_datetime_or_none(item.findtext("pracRegEndDt")),
            "pracVacancyStart": to_datetime_or_none(item.findtext("pracRegStartDt2")),
            "pracVacancyEnd": to_datetime_or_none(item.findtext("pracRegEndDt2")),
            "pracExamStart": to_datetime_or_none(item.findtext("pracExamStartDt")),
            "pracExamEnd": to_datetime_or_none(item.findtext("pracExamEndDt")),
            "pracPass": to_datetime_or_none(item.findtext("pracPassDt")),
        })
    return schedules

def save_exam_schedule(schedules):
    sql = """
    REPLACE INTO date (
        jmCd, year, period,
        docRegStart, docRegEnd, docVacancyStart, docVacancyEnd,
        docExamStart, docExamEnd, docPass,
        pracRegStart, pracRegEnd, pracVacancyStart, pracVacancyEnd,
        pracExamStart, pracExamEnd, pracPass
    ) VALUES (
        %(jmCd)s, %(year)s, %(period)s,
        %(docRegStart)s, %(docRegEnd)s, %(docVacancyStart)s, %(docVacancyEnd)s,
        %(docExamStart)s, %(docExamEnd)s, %(docPass)s,
        %(pracRegStart)s, %(pracRegEnd)s, %(pracVacancyStart)s, %(pracVacancyEnd)s,
        %(pracExamStart)s, %(pracExamEnd)s, %(pracPass)s
    )
    """
    for sch in schedules:
        cursor.execute(sql, sch)
    conn.commit()

cursor.execute("SELECT JMCD FROM qualification WHERE JMCD IS NOT NULL")
jmcd_list = [row[0] for row in cursor.fetchall()]

for jmCd in jmcd_list:
    print(f"📡 jmCd={jmCd} 시험 일정 가져오는 중...")
    schedules = fetch_exam_schedule(jmCd, "2025")
    if schedules:
        save_exam_schedule(schedules)

print("✅ 모든 자격증 시험 일정 저장 완료")

cursor.close()
conn.close()


📡 jmCd=0011 시험 일정 가져오는 중...
📡 jmCd=0012 시험 일정 가져오는 중...
📡 jmCd=0050 시험 일정 가져오는 중...
📡 jmCd=0060 시험 일정 가져오는 중...
📡 jmCd=0071 시험 일정 가져오는 중...
📡 jmCd=0080 시험 일정 가져오는 중...
📡 jmCd=0094 시험 일정 가져오는 중...
📡 jmCd=0110 시험 일정 가져오는 중...
📡 jmCd=0120 시험 일정 가져오는 중...
📡 jmCd=0130 시험 일정 가져오는 중...
📡 jmCd=0210 시험 일정 가져오는 중...
📡 jmCd=0230 시험 일정 가져오는 중...
📡 jmCd=0250 시험 일정 가져오는 중...
📡 jmCd=0261 시험 일정 가져오는 중...
📡 jmCd=0270 시험 일정 가져오는 중...
📡 jmCd=0301 시험 일정 가져오는 중...
📡 jmCd=0320 시험 일정 가져오는 중...
📡 jmCd=0360 시험 일정 가져오는 중...
📡 jmCd=0370 시험 일정 가져오는 중...
📡 jmCd=0390 시험 일정 가져오는 중...
📡 jmCd=0400 시험 일정 가져오는 중...
📡 jmCd=0410 시험 일정 가져오는 중...
📡 jmCd=0420 시험 일정 가져오는 중...
📡 jmCd=0430 시험 일정 가져오는 중...
📡 jmCd=0431 시험 일정 가져오는 중...
📡 jmCd=0432 시험 일정 가져오는 중...
📡 jmCd=0451 시험 일정 가져오는 중...
📡 jmCd=0460 시험 일정 가져오는 중...
📡 jmCd=0470 시험 일정 가져오는 중...
📡 jmCd=0480 시험 일정 가져오는 중...
📡 jmCd=0481 시험 일정 가져오는 중...
📡 jmCd=0482 시험 일정 가져오는 중...
📡 jmCd=0490 시험 일정 가져오는 중...
📡 jmCd=0501 시험 일정 가져오는 중...
📡 jmCd=0502 시험 일정 가져오는 중...
📡 jmCd=0510 시험 일정 가져